In [62]:
import pandas as pd

#import data
crypto_file_path = '~/Downloads/all.csv'
crypto_data = pd.read_csv(crypto_file_path,sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
crypto_data.columns

Index(['Unix Timestamp', 'Date', 'Symbol', 'Open', 'High', 'Low', 'Close',
       'Volume'],
      dtype='object')

In [63]:
crypto_data.head(5)

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1605571320000,2020-11-17 00:02:00,BTCUSD,16715.34,16715.34,16714.7,16714.7,0.01558624
1,1605571260000,2020-11-17 00:01:00,BTCUSD,16731.26,16731.26,16712.35,16715.34,0.05868701
2,1605571200000,2020-11-17 00:00:00,BTCUSD,16724.83,16736.79,16724.83,16731.26,0.20284979
3,1605571140000,2020-11-16 23:59:00,BTCUSD,16732.39,16732.39,16715.95,16724.83,0.7272733
4,1605571080000,2020-11-16 23:58:00,BTCUSD,16740.53,16740.53,16721.64,16732.39,2.31386403


In [64]:
#drop missing values
crypto_data = crypto_data.dropna(axis=0)

In [65]:
#select target
y = crypto_data.High

In [73]:
#select features
crypto_features = ['Open','Low','Close','Volume']
X = crypto_data[crypto_features]

In [74]:
#aimes la vie avant qu'elle te mange

In [75]:
#Checking the Data
X.describe()

,Open,Low,Close,Volume
count,1751121,1751121,1751121,1751121
unique,292559,275846,292563,665928
top,52.25,52.25,52.25,0.0
freq,1245,1246,1245,962995


In [82]:
X.head(5)

,Open,Low,Close,Volume
0,16715.34,16714.7,16714.7,0.01558624
1,16731.26,16712.35,16715.34,0.05868701
2,16724.83,16724.83,16731.26,0.20284979
3,16732.39,16715.95,16724.83,0.7272733
4,16740.53,16721.64,16732.39,2.31386403


In [85]:
#spliting the Data
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

Building the Model

1. Define model type
2. Fit the Data
3. Predict
4. Evaluate

In [86]:
from sklearn.tree import DecisionTreeRegressor

#define the model
crypto_model = DecisionTreeRegressor(random_state=1)

#fit the data
crypto_model.fit(train_X,train_y)

DecisionTreeRegressor(random_state=1)

In [87]:
print("Making predictions for the following 5 inputs:")
print(val_X.head())

Making predictions for the following 5 inputs:
           Open     Low   Close    Volume
594484   302.35  302.21  302.21       0.4
1062999   41.25   41.25   41.25       0.0
1546193   54.59   54.59   54.59       0.0
717451    195.4  195.21  195.24  9.625586
796834   227.07  227.02  228.25  1.126787


In [88]:
print("The predictions are")
print(crypto_model.predict(val_X.head()))

The predictions are
[302.24  41.25  54.59 195.4  228.25]


In [92]:
from sklearn.metrics import mean_absolute_error
#check error
val_prediction = crypto_model.predict(val_X)
val_mae = mean_absolute_error(val_y, val_prediction)
print(val_mae)

0.5773174981707528


Overfitting: too much leaves -> capturing event specific to the dataset, less accurates precisions

Underfitting: too few leaves -> failling to capture relevant patterns, less accurates precisions

To solve this, pick the right amount of leaves

In [100]:
#Define a function to test best max_leaf_node 
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    """
    return mae depending of the max_leaf_node of a tree
    """
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return mae

In [105]:
for max_leaf_nodes in [5, 50, 500, 5000,  50000, 500000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print(f'Max leaf node: {max_leaf_nodes} \t\t Mean absolute error: {my_mae}')

Max leaf node: 5 		 Mean absolute error: 197.7004714231277
Max leaf node: 50 		 Mean absolute error: 19.905301232324337
Max leaf node: 500 		 Mean absolute error: 2.5157343864642328
Max leaf node: 5000 		 Mean absolute error: 0.8060303682612393
Max leaf node: 50000 		 Mean absolute error: 0.6286487633918265
Max leaf node: 500000 		 Mean absolute error: 0.5768674786404399


The best leaf node in this cas is 500000, but after that we can't see any real improvment

The mae is 0.57687

In [106]:
#As it can be a problem to find the best max_leaf_nodes, the RandomForestRegressor is a good alternative
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
crypto_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, crypto_preds))

0.46780425753654226


The mae is 0.46780